In [1]:
from vortice.config_def import SiteConfig, GhgDataset, GhgDataField, Anemometer3DConfig, GasAnalyzerConfig


cachoeira = SiteConfig(
    name="CS", lat=-30.446667, lon=-53.194444, alt=73.0, canopy_height=0.1
)

csat_3d = Anemometer3DConfig(
    name="CSAT3D", model="CSAT3_1", height=3.0, north_offset=90.0
)

li_7500a = GasAnalyzerConfig(name="LI7500A", model="li7500a")
li_7700 = GasAnalyzerConfig(name="LI7700", model="li7700")

fields = [
    GhgDataField(name="u", variable="wind_u", unit="m/s", instrument=csat_3d),
    GhgDataField(name="v", variable="wind_v", unit="m/s", instrument=csat_3d),
    GhgDataField(name="w", variable="wind_w", unit="m/s", instrument=csat_3d),
    GhgDataField(name="TS", variable="temperature", unit="C", instrument=csat_3d),
    GhgDataField(name="diag_CSAT", variable="diagnostic", instrument=csat_3d),
    GhgDataField(name="CO2", variable="co2", unit="mmol/m^3", instrument=li_7500a),
    GhgDataField(name="H2O", variable="h2o", unit="mmol/m^3", instrument=li_7500a),
    GhgDataField(name="Diag_LI7500A", variable="diagnostic", instrument=li_7500a),
    GhgDataField(name="Temp_LI7500A", variable="temperature", unit="C", instrument=li_7500a),
    GhgDataField(name="Pressao_LI7500A", variable="pressure", unit="kPa", instrument=li_7500a),
    GhgDataField(name="Diag_LI7700", variable="diagnostic", instrument=li_7700),
    GhgDataField(name="CH4", variable="ch4", unit="mmol/m^3", instrument=li_7700),
    GhgDataField(name="CH4_mole_fraction", variable="ch4_frac", instrument=li_7700),
    GhgDataField(name="Temp_LI7700", variable="temperature", unit="C", instrument=li_7700),
    GhgDataField(name="Pressao_LI7700", variable="pressure", unit="kPa", instrument=li_7700),
    GhgDataField(name="RSSI", variable="diagnostic", instrument=li_7700),
    GhgDataField(name="Drop_rate", variable="diagnostic", instrument=li_7700),
]

CS130 = GhgDataset(name="CS130", site=cachoeira, columns=fields, time_column="TIMESTAMP", tz="UTC", freq=10)

In [2]:
from vortice.config import FluxDataFrame
import duckdb

con = duckdb.connect(database=":memory:", read_only=False)

query = "SELECT * exclude(month, year, site) FROM parquet_scan('../../dados_lumet/site=CS130/**/*.parquet', hive_partitioning = true) WHERE year=2023 AND month=12 AND TIMESTAMP>'2023-12-01 12:00:00' AND TIMESTAMP<='2023-12-01 13:00:00' order by TIMESTAMP;"
result = con.execute(query)

df_hf = (
    result.pl()
    .sort("TIMESTAMP")
    .upsample(time_column="TIMESTAMP", every="100ms", maintain_order=True)
)

In [3]:
df_hf.flux.config(CS130)